In [6]:
youtube_api_key = open('./../../API_keys/youtube.txt').read()

In [7]:
import requests

# Function to test if the YouTube API key is valid
def is_youtube_api_key_valid(api_key):
    url = "https://www.googleapis.com/youtube/v3/videos"
    params = {
        'part': 'snippet',
        'chart': 'mostPopular',
        'regionCode': 'US',
        'key': api_key
    }

    response = requests.get(url, params=params)
    return response#.status_code == 200

# Usage
#youtube_api_key = 'YOUR_YOUTUBE_API_KEY'
if is_youtube_api_key_valid(youtube_api_key):
    print("YouTube API key is valid.")
else:
    print("YouTube API key is invalid or expired.")


YouTube API key is valid.


In [8]:
is_youtube_api_key_valid(youtube_api_key)

<Response [200]>

In [9]:
import requests

def get_video_ids(api_key, channel_id):
    url = "https://www.googleapis.com/youtube/v3/search"
    video_ids = []
    next_page_token = None

    while True:
        params = {
            'part': 'id',
            'channelId': channel_id,
            'maxResults': 50,
            'pageToken': next_page_token,
            'type': 'video',
            'key': api_key
        }

        response = requests.get(url, params=params).json()

        video_ids += [item['id']['videoId'] for item in response.get('items', [])]

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

    return video_ids


In [25]:
def get_video_title(video_id, api_key):
    url = f"https://www.googleapis.com/youtube/v3/videos?id={video_id}&key={api_key}&part=snippet"
    response = requests.get(url).json()
    title = response['items'][0]['snippet']['title']
    return title

In [15]:
list_video_IDs = get_video_ids(youtube_api_key, "UCv4VkfbX8YfqodF-4coEEfQ")
print(len(list_video_IDs))
list_video_IDs[0:5]

82


['W84ws9AazSc', 'VNvH3a6Aenw', '4wi49P-Qjcc', 'XoUR_PQIdRg', '0Vijus_c-aY']

In [19]:
!pip install youtube-transcript-api

In [20]:
from youtube_transcript_api import YouTubeTranscriptApi, NoTranscriptFound

def get_transcript(video_id):
    try:
        transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
        return " ".join([item['text'] for item in transcript_list])
    except NoTranscriptFound:
        return "No transcript found"

# Example Usage
# video_ids = get_video_ids(your_api_key, channel_id)
# for video_id in video_ids:
#     print(get_transcript(video_id))


In [23]:
dict_transcripts_two = {}

for str_video_id in list_video_IDs:
    try:
        dict_transcripts_two[str_video_id] = get_transcript(str_video_id)
    except:
        print("transcript for video: '" + str_video_id + "' was unavailable")

# video_transcripts now contains a dictionary where keys are video IDs and values are transcripts


transcript for video: 'H65WG2s4pzY' was unavailable


In [34]:
dict_transcripts_two = {}

for str_video_id in list_video_IDs:
    try:
        dict_transcripts_two[str_video_id] = {'title': get_video_title(str_video_id, youtube_api_key), 
                                          'transcript': get_transcript(str_video_id)}
    except:
        print("Info for video: '" + str_video_id + "' was unavailable")

# video_transcripts now contains a dictionary where keys are video IDs and values are transcripts


Info for video: 'H65WG2s4pzY' was unavailable


In [35]:
print(len(dict_transcripts_two))
#dict_transcripts_two

81


In [36]:
print(len(dict_transcripts))
#dict_transcripts

71


In [40]:
for video_id, details in dict_transcripts_two.items():
    #print(video_id, "BLAH", details)

SyntaxError: unexpected EOF while parsing (<ipython-input-40-ad41afd879e8>, line 2)

In [42]:
import os

for video_id, details in dict_transcripts_two.items():
    title = details['title']
    transcript = details['transcript']
    
    # Replace characters not allowed in file names
    filename = "".join([c for c in title if c.isalpha() or c.isdigit() or c==' ']).rstrip()
    
    # Limiting filename length to avoid errors on some file systems
    filename = filename[:100] if len(filename) > 100 else filename

    # Writing to file
    with open(f"./data/{filename}.txt", "w", encoding="utf-8") as file:
        file.write(transcript)


In [43]:
for video_id, deets in dict_transcripts_two.items():
    deetslen(deets["transcript"].split())

SyntaxError: unexpected EOF while parsing (<ipython-input-43-ba632ff038f0>, line 1)

In [55]:
int_words_total = 0
for str_key_id in dict_transcripts_two.keys():
    #dict_transcripts_two[str_key_id]["word_count"] = len(dict_transcripts_two[str_key_id]["transcript"].split())
    #print(dict_transcripts_two[str_key_id]["word_count"])
    count = dict_transcripts_two[str_key_id]["word_count"]
    if count < 5000:
        print(count, dict_transcripts_two[str_key_id]["title"])
    int_words_total += dict_transcripts_two[str_key_id]["word_count"]

79 Neil Gaiman: A straight author with amazing queer characters?
136 The Real Burden of Being Rich
4414 The Troubling Thirst for Jeffrey Dahmer
3679 The Traumatic Camp of "Mommie Dearest"
3462 The Secret Crimes of a Dying Franchise
3956 The Gay Horror Manga You Should Be Reading
4639 When Hollywood Came Out of the Closet
4615 America v. Homosexuality
4451 Where The "Bury Your Gays" Trope Came From
4294 How a Gay Show Changed TV... and Was Forgotten
4683 Hollywood's Golden Age (of Queer Coding)
4937 How Hollywood was Born Gay
10 Coming This Fall
35 Fistory!
89 The Magic Realism of Revolutionary Girl Utena
185 Religion and Anime!
127 The Gay Horror Manga You should Be Reading - The Summer Hikaru Died #horrorstories #manga
3421 Heartstopper and Queer Optimism
2510 Harry Potter and The Closet Under The Stairs - Queer themes in Harry Potter (Video essay)
4445 The Queer Joy of Everything Everywhere All At Once
889 Geek Movie Review! Captain America: The Winter Solider
150 The Barbie to Evang

In [60]:
int_sentence_total = 0
for str_key_id in dict_transcripts_two.keys():
    dict_transcripts_two[str_key_id]["sentence_count"] = len(dict_transcripts_two[str_key_id]["transcript"].split("."))
    #print(dict_transcripts_two[str_key_id]["word_count"])
    count = dict_transcripts_two[str_key_id]["sentence_count"]
    if count > 10:
        print(count, dict_transcripts_two[str_key_id]["title"])
    int_sentence_total += dict_transcripts_two[str_key_id]["sentence_count"]

405 The Brilliance of Our Flag Means Death
201 The Secret Crimes of a Dying Franchise
221 The Gay Horror Manga You Should Be Reading
531 The Tragedy of Being Rich | James Somerton
327 The Dangers of Blissful Ignorance
345 The Real Hogwarts Legacy
231 How Hollywood was Born Gay
497 The Sadism of Class
382 For The Love of Gay Nuance
290 Disney's War Against Gay kids | James Somerton
590 SHIPPING - The Good, The Bad, and the Thirsty
491 How Wanda Became An Accidental Gay Icon
500 The Gay Appeal of Toxic Love
390 Hollywood's (Gay) China Problem | James Somerton
318 The Queer Dystopia of the LGB Movement
551 An Over-Emotional Look at Why JK Rowling is Bad
330 Disney's Gay Cultural Appropriation | James Somerton
12 Why Bad Gays Are Good
480 The Necessity of Gay Crime | James Somerton
183 Heartstopper and Queer Optimism
371 The Diversity of "The Rings of Power"
233 Disney's Silence on Gay Youth
27 Making It Big: The History of Gay Adult Film (Documentary)


In [60]:
int_sentence_total = 0
for str_key_id in dict_transcripts_two.keys():
    dict_transcripts_two[str_key_id]["sentence_count"] = len(dict_transcripts_two[str_key_id]["transcript"].split("."))
    #print(dict_transcripts_two[str_key_id]["word_count"])
    count = dict_transcripts_two[str_key_id]["sentence_count"]
    if count > 10:
        print(count, dict_transcripts_two[str_key_id]["title"])
    int_sentence_total += dict_transcripts_two[str_key_id]["sentence_count"]

405 The Brilliance of Our Flag Means Death
201 The Secret Crimes of a Dying Franchise
221 The Gay Horror Manga You Should Be Reading
531 The Tragedy of Being Rich | James Somerton
327 The Dangers of Blissful Ignorance
345 The Real Hogwarts Legacy
231 How Hollywood was Born Gay
497 The Sadism of Class
382 For The Love of Gay Nuance
290 Disney's War Against Gay kids | James Somerton
590 SHIPPING - The Good, The Bad, and the Thirsty
491 How Wanda Became An Accidental Gay Icon
500 The Gay Appeal of Toxic Love
390 Hollywood's (Gay) China Problem | James Somerton
318 The Queer Dystopia of the LGB Movement
551 An Over-Emotional Look at Why JK Rowling is Bad
330 Disney's Gay Cultural Appropriation | James Somerton
12 Why Bad Gays Are Good
480 The Necessity of Gay Crime | James Somerton
183 Heartstopper and Queer Optimism
371 The Diversity of "The Rings of Power"
233 Disney's Silence on Gay Youth
27 Making It Big: The History of Gay Adult Film (Documentary)


In [48]:
int_words_total

509802

In [58]:
int_sentence_total

8110

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [ ]:
df = pd.read_csv('./data/.csv')


In [ ]:
# Rename Column Options
new_names = ['list of new names in exact order']
new_names = {'old_name': 'new_name'}
df.columns = new_names

df.rename({
    'Old Name' : 'New_name'
}, axis = 1, inplace = True)

In [ ]:
df[''].fillna()

In [ ]:
mask = df['column'] <>!= something
df = df[mask]

In [ ]:
df.drop([], axis = 1)#inplace = True)

In [ ]:
def map_funct(cell):
    return cell

In [ ]:
df[''] = df[''].map(lambda x:)#map_funct(x))

In [ ]:
#org_qualitative_col  = df['org_qualitative_col']
df = pd.get_dummies(df)
#df['org_qualitative_col'] = org_qualitative_col

In [ ]:
# Natural Language Processing libraries, initiations and functions
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.decomposition import LatentDirichletAllocation
import re # Delete this if scraping in same notebook

def preprocess(text):
    text = re.sub(r'[^a-zA-Z]',' ', text.lower())
    tokens = word_tokenize(text)
    lemmer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    return " ".join([lemmer.lemmatize(word) for word 
                     in tokens if len(word) > 1 and not word in stop_words])
cvec = CountVectorizer(analyzer = "word",
                       min_df = 2,
                       preprocessor = preprocess,
                       stop_words = 'english')
#pd.DataFrame(cvec.fit_transform(df['COLUMN']).todense(), columns=cvec.get_feature_names())

def LDA(input_item, num_topics = 3, num_words = 5, pre_cveced = False):
    # Cols are the words. Rows are the topics
    topic_lists = []
    lda = LatentDirichletAllocation(n_components=num_topics, learning_method='online')
    
    if  pre_cveced == False: # For inserting a column and automatically cvecing things in function.
        lda.fit(cvec.fit_transform(input_item))
    elif pre_cveced == True: # For inserting a pre-cveced dataframe.
        lda.fit(input_item)
    else:
        print("Not good pre_cveced option given.")
    for ix, topic in enumerate(lda.components_):
        topic_lists += [[cvec.get_feature_names()[i] for i 
                         in lda.components_[ix].argsort()[:-num_words - 1:-1]]]

    return pd.DataFrame(topic_lists, columns=[ 'word_' + str(i) for i 
                                    in range(1, num_words+1)], index=range(1, num_topics + 1)) 


def Sentamentize(text):
    return TextBlob(str(text)).sentiment.polarity

In [ ]:

df.dtypes# Return column data types
df.describe() #(returns count, mean, std, min, and %iles)???
df.info()
df['col1'].std() # Standard Deviation


# Main ways to adjust indexes
df.reset_index(inplace=True)
df.set_index('column', inplace = True)
df = df.reindex(['cols'], axis = 1)



# Concat
the_dfs = [df_one,df_two]
df_main = pd.concat(the_dfs, sort=False, axis = 1)

# Dropping columns with "Errors"
mask = df['col'] == 'Error'
df.drop(df[mask].index, inplace = True)

# Dropping Duplicates
df.drop_duplicates(inplace = True)

# Map Function Starter
def map_funct(cell):
    return cell
df['column'] = df['column'].map(lambda x: map_funct(x))


# Masks
mask = df['column'] <=> something 
df = df[mask]

df[~mask] (Select opposite of the mask)
df[(df['col1'] > num) & (df['col2'] < num)]
df[(df['col1'] > num) | (df['col2'] > num)]
df[ df['col'].isin([num, num]) ] #return rows with specific values(?)

### EDA

In [ ]:
# Basic EDA

df.dtypes                           # Data Types of cols
df.column.unique()                  # Unique Values in col
df.describe().T                     # Summary Stats
df.info                             # Info
df['column'].value_counts(normalize=True)

In [ ]:
| Thing             | Main Matplotlib               | Sub Plt               | Seaborn                           |
|-------------------|-------------------------------|-----------------------|-----------------------------------|
| Idea              | + to plt each line            |fig, ax = plt.subplots(ncols, nrows) (ax[#,#] for ea plt)| |
| Bar               | plt.bar(x, height)            |                       |                                   |
| Histagram         | plt.hist(x, bins)(Lists Args) |                       |                                   |
| Scatter           | plt.scatter(x, y)    (Args)   |                       |                                   |
| Line Graph        | plt.plot([y's_ea_x(#,#)(#,])  | ax[#]plot()           | sns.lineplot(x,y)(Args)           |
| FacetGrid (Chi2)  |                               |                       | sns.FacetGrid(df, col, row)       |
| ...define inr plts|                               |                       |.map(plt.graph, bins, range)       |
| X/Y Labels        | plt.zlabel("")                | ax[#].set_xlabel()    |                                   |
| Title             | plt.title("")                 | ax[#].set_title()     |                                   |
| Figure Size       | plt.figure(figsize)           | plt.subplots(figsize) |                                   |
|Customize X/Y Ticks| plt.zticks([labels], rotation)|
| Add a Line        | plt.axvline(x)                |
| Set Z Value Limit | plt.gca().set_zlim([#,#])     |
| Add Legend        | plt.legend()                  |


The six "lessons" of visualizing data.
 - Lesson 1: Understand the context. (Who, What, How)
 - Lesson 2: Choose an appropriate visual display. (What type of graph?)
 - Lesson 3: Eliminate clutter. (What can I delete or make more subtle?)
 - Lesson 4: Focus attention where you want it. (How do I emphasize important things?)
 - Lesson 5: Think like a designer. (How do I organize my visualization?)
 - Lesson 6: Tell a story! (How do I communicate my visualization?)


-------------------------------------------

## Set Up Options

# import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Color Options
colors = {'blue': '#729ECE',
          'brown': '#A8786E',
          'green': '#67BF5C',
          'grey': '#A2A2A2',
          'orange': '#FF9E4A',
          'pink': '#ED97CA',
          'purple': '#AD8BC9',
          'red': '#ED665D',
          'teal': '#6DCCDA',
          'yellow': '#CDCC5D'}

# Style Options
sns.set_style('darkgrid')
sns.set_style('whitegrid')
plt.style.use('fivethirtyeight')


# Basic Non-Visual EDA

df.dtypes                           # Data Types of cols
df.column.unique()                  # Unique Values in col
df.describe().T                     # Summary Stats
df.info                             # Info
df['column'].value_counts(normalize=True)

# Correlations between two seperate columns w/o a whole thing
print(np.corrcoef(df['column_1'].values, df['column_2'].values))

# SCATTER


## PLT
plt.figure(figsize=(10, 10))
plt.scatter(y = df['column_one'],
            x = df['column_two'],
            color = colors);

## PD
df.plot('math', 'num_awards', kind = 'scatter')


## SNS Dirty
axes = plt.gca()
axes.set_xlim([-1,35])
axes.set_ylim([-1,35])
sns.scatterplot(df['column_one'], df['column_two'], hue= df['column_three'], alpha = 0.1);

# BOXPLOT

## SNS
sns.boxplot(df['column'],
            data = df
                orient='v' or 'h',
                fliersize=8,
                linewidth=#,
                notch = True,
                saturation=0.5,
                ax=ax,
                by = "column?");

# LINEPLOT

sns.lineplot(data = df);

====================================================
# HISTAGRAM

## PLT
plt.hist(df['column'], bins = 10, color='g', alpha = 0.1);
plt.xlabel("Label", position = (0,0), ha = 'left', color = 'grey') # (x,y)
plt.ylabel("label", position = (0, 1), ha = 'right', color = 'grey'); # (x,y);

## PLT.plot
figure, ax = plt.subplots(nrows = 2, ncols = 1, figsize = (15,15))
test_df['column_one'].plot(ax = ax[0], kind = 'hist')
test_df['column_two'].plot(ax = ax[1], kind = 'hist');

## PD
df.plot(x='col1', y='col2', title="Title")
df.hist('col');

## SNS Countplot
sns.countplot(raw_df['col']);
##Also
sns.countplot(x="col", data=df);

------------
# Other Example DIRTY
plt.hist(df['col1'],bins= len(df['col1'].unique()));


====================================================
# BARPLOT

## PLT
plt.figure(figsize = (16,6))
plt.bar(height = df['Column?'],  # List of numbers?
        x = df.columns);

## SNS
sns.barplot(data = df,
            x = "column_one",
            y = "column_two");




====================================================
# PAIRPLOT

sns.pairplot(df, kind = "reg", 
             hue = 'column',
             plot_kws = {'line_kws' : {'color' : 'red'}, 
                         'scatter_kws' : {'alpha' : 0.1}})
             # Alt
             # plot_kws={'alpha': 0.05});

# With specific rows, only
sns.pairplot(data=temp_df,
            x_vars=['col1'],
            y_vars=['col2'])


====================================================
# HEATMAP

# SNS
#plt.figure(figsize= (9,9))
sns.heatmap(df.corr(), 
            cbar=False, 
            linewidths= .01, 
            linecolor='black', 
            cmap=sns.color_palette("Purples"),
            annot=True);


=======================================================
# Jointplot
sns.jointplot(y_test, lin_reg.predict(X_test))

=======================================================
# Facet Grid
Grid = sns.FacetGrid(df, col="Col1", row="col2", margin_titles=True)
Grid.map(plt.hist,'col2?', bins = num, range = (num,num));

=======================================================
# Graph Alterations
# Colors: plt.colors.cnames

# For layering on top:
plt.hist(df['column_1'], color='g')
plt.hist(df['column_2'], color='b')

# For layering next to:
fig, ax = plt.subplots(2, 
                    sharex = True,  # Scale x together
                    sharey = True,    
                   figsize = (1,1)) # Size
ax[0].hist(df['column_1'],          # 
           bins = 10,               # Bins
           color='b',               # Color
           alpha = 0.5,             # Transparancy
           label = 'col_1')         # For .legend()
ax[1].hist(df['column_2'])          # 

# Subplot Example

fig, axes = plt.subplots(3, figsize = (20,10))
sns.countplot(df['col'], ax= axes[0])
sns.countplot(df['col'], ax = axes[1])
sns.countplot(df['col'], ax = axes[2]);

# Labels for Words
plt.xlabel("measure",               # Alt: .ylabel
           position = (0,0),        # (x-pos, y-pos)?
           ha = 'left',             # Horizontal
           color = 'grey') # (x,y)

plt.legend()
plt.title()                         # Title ~= Subtitle
plt.suptitle("subtitle\n.",         # Subtitle
             position = (0,1),      # Position
             ha = 'left',           # Horizontal
             fontsize=16,           # Size, Font
             va = 'top');           # Vertical


# Shifting Graph Coordinates (in a Scatter plot)
axes = plt.gca()
axes.set_xlim([num_one,num_two])
axes.set_ylim([num_one,num_two])
sns.scatterplot(df['column_one'], df['column_two']);